In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

from datetime import datetime, timedelta
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.preprocessing import StandardScaler

import os
import sys
import logging
from logging.handlers import TimedRotatingFileHandler

In [2]:
ALGORITHM = "knn"
RANDOM_SEED = 42
TAMANHO_DA_JANELA_EM_DIAS = 7
TIPO_ILUMINANCIA = "total"

LOG_SAVE_PATH = "7_logs_cenario_real"
LOG_FILENAME = f"{ALGORITHM}_{TAMANHO_DA_JANELA_EM_DIAS}_dias_iluminancia_{TIPO_ILUMINANCIA}.log"

RESULTS_SAVE_PATH = "7_logs_cenario_real"
DAILY_RESULTS_FILENAME = f"{ALGORITHM}_{TAMANHO_DA_JANELA_EM_DIAS}_dias_iluminancia_{TIPO_ILUMINANCIA}.csv"
RESULTS_FILENAME = f"{ALGORITHM}_iluminancia_{TIPO_ILUMINANCIA}.csv"

In [3]:
def log_setup(filename, log_level):
    logger = logging.getLogger(filename)
    logger.setLevel(log_level)
    formatter = logging.Formatter(fmt='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                                  datefmt='%m-%d-%y %H:%M:%S')
    fh = TimedRotatingFileHandler(filename, when='midnight')
    fh.setFormatter(formatter)
    sh = logging.StreamHandler(sys.stdout)
    sh.setLevel(log_level)
    sh.setFormatter(formatter)
    logger.addHandler(fh)
    logger.addHandler(sh)
    return logger

In [4]:
os.makedirs(LOG_SAVE_PATH, exist_ok=True)
if os.path.isfile(f"{LOG_SAVE_PATH}/{LOG_FILENAME}"):
    os.remove(f"{LOG_SAVE_PATH}/{LOG_FILENAME}")
logger = log_setup(f"{LOG_SAVE_PATH}/{LOG_FILENAME}", logging.INFO)

In [5]:
data = pd.read_csv("../mai_2022_fev_2023.csv")

In [6]:
len(data)

81501

In [7]:
data.head()

,iluminancia_caixa,iluminancia_teto,temperatura,usuario_1,usuario_2,usuario_3,usuario_4,usuario_5,usuario_6,usuario_7,...,data_ano,data_mes,data_dia,data_hora,data_minuto,data_dia_da_semana,porta,janela1,janela2,output
0,263,62,23.73,0,0,0,0,0,0,0,...,2022,6,7,14,28,1,1,1,1,0
1,25,65,24.36,0,0,0,0,0,0,0,...,2022,6,7,14,30,1,1,1,1,0
2,266,66,23.87,0,0,0,0,0,0,0,...,2022,6,7,14,32,1,1,1,1,0
3,272,68,23.87,0,0,0,0,0,0,0,...,2022,6,7,14,34,1,1,1,1,0
4,275,71,23.87,0,0,0,0,0,0,0,...,2022,6,7,14,36,1,1,1,1,0


In [8]:
data.tail()

,iluminancia_caixa,iluminancia_teto,temperatura,usuario_1,usuario_2,usuario_3,usuario_4,usuario_5,usuario_6,usuario_7,...,data_ano,data_mes,data_dia,data_hora,data_minuto,data_dia_da_semana,porta,janela1,janela2,output
81496,164,195,35.12,0,0,1,1,0,1,1,...,2023,2,16,15,1,3,1,1,1,3
81497,165,195,35.16,0,0,1,1,0,1,1,...,2023,2,16,15,6,3,1,1,1,3
81498,165,194,35.17,0,0,1,1,0,1,1,...,2023,2,16,15,12,3,1,1,1,3
81499,164,189,35.17,0,0,1,1,0,1,1,...,2023,2,16,15,17,3,1,1,1,3
81500,162,183,35.17,0,0,1,1,0,1,1,...,2023,2,16,15,22,3,1,1,1,3


In [9]:
features_names = list(data.columns)[:-1]
logger.info(f"features_names: {features_names}")
logger.info(f"len(features_names): {len(features_names)}")

11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     features_names: ['iluminancia_caixa', 'iluminancia_teto', 'temperatura', 'usuario_1', 'usuario_2', 'usuario_3', 'usuario_4', 'usuario_5', 'usuario_6', 'usuario_7', 'usuario_8', 'usuario_9', 'usuario_10', 'usuario_11', 'usuario_12', 'usuario_13', 'usuario_14', 'usuario_15', 'usuario_16', 'usuario_17', 'usuario_18', 'usuario_19', 'usuario_20', 'usuario_21', 'usuario_22', 'usuario_23', 'usuario_24', 'usuario_25', 'data_ano', 'data_mes', 'data_dia', 'data_hora', 'data_minuto', 'data_dia_da_semana', 'porta', 'janela1', 'janela2']
11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     len(features_names): 37


In [10]:
features_names_iluminancia_teto = features_names.copy()
features_names_iluminancia_caixa = features_names.copy()

features_names_iluminancia_teto.remove('iluminancia_caixa')
features_names_iluminancia_caixa.remove('iluminancia_teto')

logger.info(f"features_names_iluminancia_teto: {features_names_iluminancia_teto}")
logger.info(f"features_names_iluminancia_caixa: {features_names_iluminancia_caixa}")

11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     features_names_iluminancia_teto: ['iluminancia_teto', 'temperatura', 'usuario_1', 'usuario_2', 'usuario_3', 'usuario_4', 'usuario_5', 'usuario_6', 'usuario_7', 'usuario_8', 'usuario_9', 'usuario_10', 'usuario_11', 'usuario_12', 'usuario_13', 'usuario_14', 'usuario_15', 'usuario_16', 'usuario_17', 'usuario_18', 'usuario_19', 'usuario_20', 'usuario_21', 'usuario_22', 'usuario_23', 'usuario_24', 'usuario_25', 'data_ano', 'data_mes', 'data_dia', 'data_hora', 'data_minuto', 'data_dia_da_semana', 'porta', 'janela1', 'janela2']
11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     features_names_iluminancia_caixa: ['iluminancia_caixa', 'temperatura', 'usuario_1', 'usuario_2', 'usuario_3', 'usuario_4', 'usuario_5', 'usuario_6', 'usuario_7', 'usuario_8', 'usuario_9', 'usuario_10', 'usuario_11', 'usuario_12', 'usuario_13', 'usuario_14', 'usuario_15', 'usuario_16', 'usuario_17', 'usuario_18',

In [11]:
if TIPO_ILUMINANCIA == "teto":
    features_names = features_names_iluminancia_teto.copy()
elif TIPO_ILUMINANCIA == "caixa":
    features_names = features_names_iluminancia_caixa.copy()
elif TIPO_ILUMINANCIA == "total":
    pass
else:
    raise ValueError("Escolha entre os dois tipos de iluminâncias: 'teto' ou 'caixa'")

logger.info(f"features_names: {features_names}")

11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     features_names: ['iluminancia_caixa', 'iluminancia_teto', 'temperatura', 'usuario_1', 'usuario_2', 'usuario_3', 'usuario_4', 'usuario_5', 'usuario_6', 'usuario_7', 'usuario_8', 'usuario_9', 'usuario_10', 'usuario_11', 'usuario_12', 'usuario_13', 'usuario_14', 'usuario_15', 'usuario_16', 'usuario_17', 'usuario_18', 'usuario_19', 'usuario_20', 'usuario_21', 'usuario_22', 'usuario_23', 'usuario_24', 'usuario_25', 'data_ano', 'data_mes', 'data_dia', 'data_hora', 'data_minuto', 'data_dia_da_semana', 'porta', 'janela1', 'janela2']


In [12]:
data['date'] = pd.to_datetime(dict(year=data.data_ano, month=data.data_mes, day=data.data_dia))

In [13]:
original_data = data.copy()

In [14]:
data['date'].unique()

<DatetimeArray>
['2022-06-07 00:00:00', '2022-06-08 00:00:00', '2022-06-09 00:00:00',
 '2022-06-10 00:00:00', '2022-06-11 00:00:00', '2022-06-12 00:00:00',
 '2022-06-13 00:00:00', '2022-06-14 00:00:00', '2022-06-15 00:00:00',
 '2022-06-16 00:00:00',
 ...
 '2023-02-07 00:00:00', '2023-02-08 00:00:00', '2023-02-09 00:00:00',
 '2023-02-10 00:00:00', '2023-02-11 00:00:00', '2023-02-12 00:00:00',
 '2023-02-13 00:00:00', '2023-02-14 00:00:00', '2023-02-15 00:00:00',
 '2023-02-16 00:00:00']
Length: 253, dtype: datetime64[ns]

In [15]:
def save_or_append_avg_metrics(colunas_geral, metrics_geral, path):
    new_data = pd.DataFrame(columns=colunas_geral, data=metrics_geral)
    
    if os.path.isfile(path):
        data = pd.read_csv(path)
        data = pd.concat([data, new_data])
        data.to_csv(path, index=False)
        print(f"Arquivo já existe: {path}")
    else:
        print("Arquivo não existe")
        new_data.to_csv(path, index=False)

In [16]:
colunas = ["Teste", "Precision", "Recall", "F1", "Accuracy"]

metrics = []
precision_list = []
recall_list = []
f1_list = []
accuracy_list = []

metrics_w = []
precision_w_list = []
recall_w_list = []
f1_w_list = []
accuracy_w_list = []

i = 0

# for i in range(N_EXPERIMENTOS):
while True:
    data = original_data.copy()
    logger.info(f"==================== Teste nº{i+1} (i={i}) ====================")

    min_date = datetime.strptime(str(data['date'].min())[:10], '%Y-%m-%d')
    max_date = datetime.strptime(str(data['date'].max())[:10], '%Y-%m-%d')
    
    # Dados de treino
    # Aqui foi colocado o "+30" para começar depois da data do primeiro teste
    initial_date_train = min_date + timedelta(days = i) 
    final_date_train = initial_date_train + timedelta(days = TAMANHO_DA_JANELA_EM_DIAS)
    if final_date_train > max_date:
        logger.info("A data final de treino é maior que a data máxima")
        break

    # Dados de teste
    initial_date_test = final_date_train# + timedelta(days = 1)
    # final_date_test = initial_date_test + timedelta(days = TAMANHO_DA_JANELA_EM_DIAS)
    final_date_test = initial_date_test + timedelta(days = 1)

    if initial_date_test > max_date:
        logger.info("A data inicial de teste é maior que a data máxima")
        break
    elif final_date_test > max_date:
        logger.info("A data final de teste é maior que a data máxima")
        break
    
    logger.info(f"initial_date_train: {initial_date_train}")
    logger.info(f"final_date_train: {final_date_train}")
    logger.info(f"initial_date_test: {initial_date_test}")
    logger.info(f"final_date_test: {final_date_test}")

    data_train = data.loc[(data['date'] >= initial_date_train) & (data['date'] < final_date_train)]
    # logger.info(f"data_train: {len(data_train)}")
    data_test = data.loc[(data['date'] >= initial_date_test) & (data['date'] < final_date_test)]
    # logger.info(f"data_test: {len(data_test)}")

    logger.info(f"min_date_train: {data_train['date'].min()}")
    logger.info(f"max_date_train: {data_train['date'].max()}")
    logger.info(f"min_date_test: {data_test['date'].min()}")
    logger.info(f"max_date_test: {data_test['date'].max()}")
    
    if str(data_train['date'].min()) == "NaT" or \
        str(data_train['date'].max()) == "NaT" or \
        str(data_test['date'].min()) == "NaT" or \
        str(data_test['date'].max()) == "NaT":
        logger.info("Erro: amostras vazias.")
        i += 1
        continue

    logger.info(f"Distribuição de classes - Treino: {data_train['output'].value_counts()}")
    logger.info(f"Distribuição de classes - Teste: {data_test['output'].value_counts()}")

    X_train = data_train[features_names]
    # logger.info(f"X_train.shape: {X_train.shape}")

    X_train = X_train.values.tolist()
    y_train = data_train['output'].values.tolist()

    X_test = data_test[features_names]
    if len(X_train) < 5 or len(X_test) < 5:
        logger.info("Quantidade de amostras é menor que a quantidade de vizinhos...")
        i += 1
        continue
    
    # logger.info(f"X_test.shape: {X_test.shape}\n")

    X_test = X_test.values.tolist()
    y_test = data_test['output'].values.tolist()

    # Train-Test
    logger.info("-- Train/Test --")
    clf = KNeighborsClassifier()
    try:
        clf.fit(X_train, y_train)
    except Exception as error:
        logger.info(f"Erro: {error}")
        continue

    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    acc = accuracy_score(y_test, y_pred)

    metrics.append([i+1, precision, recall, f1, acc])
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    accuracy_list.append(acc)

    i += 1

11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     ==================== Teste nº1 (i=0) ====================
11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_train: 2022-06-07 00:00:00
11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_train: 2022-06-14 00:00:00
11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_test: 2022-06-14 00:00:00
11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_test: 2022-06-15 00:00:00
11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_train: 2022-06-07 00:00:00
11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     max_date_train: 2022-06-13 00:00:00
11-10-23 19:02:36 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_test: 2022-06-14 00:00:00
11-10-23 19:02:36 7_logs_cenario_real/knn

/home/luiz/anaconda3/envs/prime/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/luiz/anaconda3/envs/prime/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/luiz/anaconda3/envs/prime/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     ==================== Teste nº10 (i=9) ====================
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_train: 2022-06-16 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_train: 2022-06-23 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_test: 2022-06-23 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_test: 2022-06-24 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_train: 2022-06-16 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     max_date_train: 2022-06-22 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_test: 2022-06-23 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/kn

/home/luiz/anaconda3/envs/prime/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/luiz/anaconda3/envs/prime/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/luiz/anaconda3/envs/prime/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_train: 2022-06-19 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     max_date_train: 2022-06-25 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_test: 2022-06-26 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     max_date_test: 2022-06-26 00:00:00
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     Distribuição de classes - Treino: output
3    2345
0    2305
1     123
2       1
Name: count, dtype: int64
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     Distribuição de classes - Teste: output
0    448
3    249
Name: count, dtype: int64
11-10-23 19:02:37 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     -- Train/Test --
11-10-23 19:02:38 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     ==========

...

11-10-23 19:02:46 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     ==================== Teste nº218 (i=217) ====================
11-10-23 19:02:46 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_train: 2023-01-10 00:00:00
11-10-23 19:02:46 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_train: 2023-01-17 00:00:00
11-10-23 19:02:46 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_test: 2023-01-17 00:00:00
11-10-23 19:02:46 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_test: 2023-01-18 00:00:00
11-10-23 19:02:46 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_train: 2023-01-10 00:00:00
11-10-23 19:02:46 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     max_date_train: 2023-01-16 00:00:00
11-10-23 19:02:46 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_test: 2023-01-17 00:00:00
11-10-23 19:02:46 7_logs_cenario_real

/home/luiz/anaconda3/envs/prime/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/luiz/anaconda3/envs/prime/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


11-10-23 19:02:46 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     -- Train/Test --
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     ==================== Teste nº223 (i=222) ====================
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_train: 2023-01-15 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_train: 2023-01-22 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_test: 2023-01-22 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_test: 2023-01-23 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_train: 2023-01-15 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     max_date_train: 2023-01-21 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_ilumin

/home/luiz/anaconda3/envs/prime/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     ==================== Teste nº231 (i=230) ====================
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_train: 2023-01-23 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_train: 2023-01-30 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     initial_date_test: 2023-01-30 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     final_date_test: 2023-01-31 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_train: 2023-01-23 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     max_date_train: 2023-01-29 00:00:00
11-10-23 19:02:47 7_logs_cenario_real/knn_7_dias_iluminancia_total.log INFO     min_date_test: 2023-01-30 00:00:00
11-10-23 19:02:47 7_logs_cenario_real

In [17]:
colunas_geral = [
    "Janela",
    "Media_Precision",
    "Media_Recall",
    "Media_F1",
    "Media_Accuracy",
    "Desvio_Precision",
    "Desvio_Recall",
    "Desvio_F1",
    "Desvio_Accuracy"
]
metrics_geral = [
    [TAMANHO_DA_JANELA_EM_DIAS,
    np.mean(precision_list),
    np.mean(recall_list),
    np.mean(f1_list),
    np.mean(accuracy_list),
    np.std(precision_list),
    np.std(recall_list),
    np.std(f1_list),
    np.std(accuracy_list)]
]

save_or_append_avg_metrics(colunas_geral, metrics_geral, f"{RESULTS_SAVE_PATH}/{RESULTS_FILENAME}")

Arquivo não existe


# Resultados diários

In [18]:
analise_dias = pd.DataFrame(columns=colunas, data=metrics)
analise_dias.to_csv(f"{RESULTS_SAVE_PATH}/{DAILY_RESULTS_FILENAME}", index=False)